In [13]:
import pandas as pd
import numpy as np
import openpyxl
from scipy.interpolate import griddata


# Load the dataset
file_path = 'Compressor_clean.xlsx'
df = pd.read_excel(file_path, sheet_name=None)

print(df.keys())

print(df)


dict_keys(['page-1_table-1'])
{'page-1_table-1':     Unnamed: 0 Unnamed: 1     36     48     55     70     87    107     130  \
0        613.0          C    NaN    NaN    NaN    NaN    NaN    NaN  34,300   
1          NaN          P    NaN    NaN    NaN    NaN    NaN    NaN   5,230   
2          NaN          A    NaN    NaN    NaN    NaN    NaN    NaN    23.3   
3          NaN          M    NaN    NaN    NaN    NaN    NaN    NaN     594   
4          NaN          E    NaN    NaN    NaN    NaN    NaN    NaN     6.6   
..         ...        ...    ...    ...    ...    ...    ...    ...     ...   
57         NaN          P  2,200  2,190  2,170  2,140  2,080  2,020   1,960   
58         NaN          A    9.9    9.8    9.8    9.6    9.4    9.1     8.8   
59         NaN          M    209    266    298    368    450    544     654   
60         NaN          E    8.0   10.3   11.7   14.9   18.9   23.8    29.8   
61         NaN          %   57.4   63.3   65.7   69.6   71.5   70.7    66.2   

  

In [69]:
import pandas as pd
from scipy.interpolate import interp1d, interp2d
from scipy.interpolate import griddata
import numpy as np

# Load the dataset
file_path = 'Compressor_clean.xlsx'
df = pd.read_excel(file_path)

# Manually set the header based on the correct content
df.columns = ['Condensing_Temperature', 'Type', 36, 48, 55, 70, 87, 107, 130, 143, 156]

# Filter the Capacity (C) data
capacity_data = df[df['Type'] == 'C']
capacity_data = capacity_data.drop(columns=['Type']).reset_index(drop=True)

# Convert the data to numeric, handling the conversion of commas in the values
capacity_data = capacity_data.replace({',': ''}, regex=True).apply(pd.to_numeric, errors='coerce')

# Prepare data for 2D interpolation
condensing_temps = capacity_data['Condensing_Temperature'].values
evap_temps = capacity_data.columns[1:].astype(float)
capacities = capacity_data.iloc[:, 1:].values

# Create a 2D interpolation function
interp_func_2d = interp2d(evap_temps, condensing_temps, capacities, kind='linear')

# Function to interpolate Capacity values
def interpolate_capacity(evap_temp, cond_temp):
    # Check if the evaporating temperature is within the range of available data
    if evap_temp < evap_temps.min() or evap_temp > evap_temps.max():
        return np.nan

    # Check if the condensing temperature is within the range of available data
    if cond_temp < condensing_temps.min() or cond_temp > condensing_temps.max():
        return np.nan

    # Perform 2D interpolation
    interpolated_capacity = interp_func_2d(evap_temp, cond_temp)
    
    return interpolated_capacity[0]  # Extract the interpolated value

evap_temp_input = 140
cond_temp_input = 236
interpolated_capacity = interpolate_capacity(evap_temp_input, cond_temp_input)
print(interpolated_capacity)


62661.53846153847


C:\Users\bober\AppData\Local\Temp\ipykernel_8360\2427458942.py:26: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  interp_func_2d = interp2d(evap_temps, condensing_temps, capacities, kind='linear')
C:\Users\bober\AppData\Local\Temp\ipykernel_8360\2427458942.py:39: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids u